In [ ]:
import os
import autogen
import pandas as pd
import pulp
import json
import logging
import ipywidgets as widgets
from IPython.display import display, HTML, clear_output
from typing import Dict, List
import random


# Set up logging
logging.basicConfig(level=logging.INFO, format='%(asctime)s - %(levelname)s - %(message)s')

In [ ]:
class LineupOptimizerAgent(autogen.AssistantAgent):
    def __init__(self, name, data_file, llm_config):
        super().__init__(name=name, llm_config=llm_config)
        self.data = pd.read_csv(data_file)
        self.preprocess_data()
        self.previous_lineups = []

    def preprocess_data(self):
        self.data['player_name'] = self.data['player_name'].str.replace(' ', '_')
        self.data['salary'] = self.data['salary'].replace('[\$,]', '', regex=True).astype(float)
        self.data['projected_points'] = pd.to_numeric(self.data['projected_points'], errors='coerce')

    def optimize_lineup(self, constraints: Dict, diversity_constraint: Dict = None) -> Dict:
        players = self.data.to_dict('records')
        prob = pulp.LpProblem("Fantasy Football", pulp.LpMaximize)
        
        player_vars = pulp.LpVariable.dicts("players", 
                                            ((p['player_name'], p['player_position_id']) for p in players), 
                                            cat='Binary')

        # Objective: Maximize total projected points
        prob += pulp.lpSum([player['projected_points'] * player_vars[player['player_name'], player['player_position_id']] 
                            for player in players])

        # Basic constraints
        prob += pulp.lpSum([player['salary'] * player_vars[player['player_name'], player['player_position_id']] for player in players]) <= 50000
        prob += pulp.lpSum([player_vars[p['player_name'], 'QB'] for p in players if p['player_position_id'] == 'QB']) == 1
        prob += pulp.lpSum([player_vars[p['player_name'], 'RB'] for p in players if p['player_position_id'] == 'RB']) >= 2
        prob += pulp.lpSum([player_vars[p['player_name'], 'WR'] for p in players if p['player_position_id'] == 'WR']) >= 3
        prob += pulp.lpSum([player_vars[p['player_name'], 'TE'] for p in players if p['player_position_id'] == 'TE']) == 1
        prob += pulp.lpSum([player_vars[p['player_name'], 'DST'] for p in players if p['player_position_id'] == 'DST']) == 1
        prob += pulp.lpSum([player_vars[p['player_name'], p['player_position_id']] for p in players]) == 9

        # Apply user-defined constraints
        if 'must_include' in constraints:
            for player_name in constraints['must_include']:
                prob += pulp.lpSum([player_vars[p['player_name'], p['player_position_id']] 
                                    for p in players if p['player_name'].lower() == player_name.lower()]) == 1

        if 'exclude' in constraints:
            for player_name in constraints['exclude']:
                prob += pulp.lpSum([player_vars[p['player_name'], p['player_position_id']] 
                                    for p in players if p['player_name'].lower() == player_name.lower()]) == 0

        # Apply diversity constraint
        if diversity_constraint:
            prob += pulp.lpSum([player_vars[p['player_name'], p['player_position_id']] 
                                for p in players if p['player_name'] in diversity_constraint['players']]) <= diversity_constraint['max_players']

        # Solve the problem
        prob.solve()

        if pulp.LpStatus[prob.status] != 'Optimal':
            return {'error': f"Optimization failed: {pulp.LpStatus[prob.status]}"}

        return self.build_lineup(player_vars, players)

    def build_lineup(self, player_vars, players):
        lineup = {
            'QB': None, 'RB1': None, 'RB2': None, 'WR1': None, 'WR2': None, 'WR3': None, 
            'TE': None, 'FLEX': None, 'DST': None
        }

        for player in players:
            if player_vars[player['player_name'], player['player_position_id']].value() == 1:
                position = player['player_position_id']
                if position == 'QB' and lineup['QB'] is None:
                    lineup['QB'] = player
                elif position == 'RB':
                    if lineup['RB1'] is None:
                        lineup['RB1'] = player
                    elif lineup['RB2'] is None:
                        lineup['RB2'] = player
                    elif lineup['FLEX'] is None:
                        lineup['FLEX'] = player
                elif position == 'WR':
                    if lineup['WR1'] is None:
                        lineup['WR1'] = player
                    elif lineup['WR2'] is None:
                        lineup['WR2'] = player
                    elif lineup['WR3'] is None:
                        lineup['WR3'] = player
                    elif lineup['FLEX'] is None:
                        lineup['FLEX'] = player
                elif position == 'TE':
                    if lineup['TE'] is None:
                        lineup['TE'] = player
                    elif lineup['FLEX'] is None:
                        lineup['FLEX'] = player
                elif position == 'DST':
                    lineup['DST'] = player

        return lineup

    def generate_lineups(self, constraints: Dict) -> List[Dict]:
        num_lineups = constraints.get('num_lineups', 1)
        lineups = []
        for i in range(num_lineups):
            if i == 0:
                lineup = self.optimize_lineup(constraints)
            else:
                # Create diversity constraint based on previous lineups
                previous_players = set(player['player_name'] for lineup in lineups for player in lineup.values() if player)
                diversity_constraint = {
                    'max_players': max(5 - i, 2),  # More aggressive reduction
                    'players': previous_players
                }
                
                # Temporarily exclude some random players to force diversity
                excluded_players = random.sample(list(previous_players), min(3, len(previous_players)))
                temp_constraints = constraints.copy()
                temp_constraints['exclude'] = excluded_players
                
                lineup = self.optimize_lineup(temp_constraints, diversity_constraint)
            
            if 'error' in lineup:
                return [lineup]  # Return error if optimization fails
            lineups.append(lineup)
            self.previous_lineups.append(lineup)
        return lineups

In [ ]:
def create_fantasy_football_ui(optimizer_agent, user_input_agent, user_proxy):
    input_text = widgets.Text(
        value='',
        placeholder='Enter your lineup request',
        description='Request:',
        disabled=False
    )

    generate_button = widgets.Button(
        description='Generate Lineup(s)',
        disabled=False,
        button_style='success',
        tooltip='Click to generate lineup(s)'
    )

    output = widgets.Output()

    prev_button = widgets.Button(description='Previous', disabled=True)
    next_button = widgets.Button(description='Next', disabled=True)
    lineup_number = widgets.Label(value='Lineup 1 of 1')

    def display_lineup(lineup):
        html = """
        <style>
            .lineup-table { border-collapse: collapse; width: 100%; }
            .lineup-table th, .lineup-table td { border: 1px solid #ddd; padding: 8px; text-align: left; }
            .lineup-table tr:nth-child(even) { background-color: #f2f2f2; }
            .lineup-table th { background-color: #4CAF50; color: white; }
        </style>
        <table class="lineup-table">
            <tr>
                <th>Position</th>
                <th>Player</th>
                <th>Projected Points</th>
                <th>Salary</th>
            </tr>
        """
        total_salary = 0
        total_points = 0
        for position, player in lineup.items():
            if player:
                html += f"""
                <tr>
                    <td>{position}</td>
                    <td>{player['player_name']}</td>
                    <td>{player['projected_points']:.2f}</td>
                    <td>${player['salary']}</td>
                </tr>
                """
                total_salary += player['salary']
                total_points += player['projected_points']
        html += f"""
            <tr>
                <th colspan="2">Totals</th>
                <th>{total_points:.2f}</th>
                <th>${total_salary}</th>
            </tr>
        </table>
        """
        return html

    current_lineup = 0
    lineups = []

    def update_buttons():
        prev_button.disabled = current_lineup == 0
        next_button.disabled = current_lineup == len(lineups) - 1
        lineup_number.value = f'Lineup {current_lineup + 1} of {len(lineups)}'

    def on_prev_clicked(b):
        nonlocal current_lineup
        if current_lineup > 0:
            current_lineup -= 1
            with output:
                clear_output()
                display(HTML(display_lineup(lineups[current_lineup])))
            update_buttons()

    def on_next_clicked(b):
        nonlocal current_lineup
        if current_lineup < len(lineups) - 1:
            current_lineup += 1
            with output:
                clear_output()
                display(HTML(display_lineup(lineups[current_lineup])))
            update_buttons()

    prev_button.on_click(on_prev_clicked)
    next_button.on_click(on_next_clicked)

    def on_generate_button_clicked(b):
        nonlocal lineups, current_lineup
        with output:
            clear_output()
            print("Generating lineup(s)...")
            constraints = {}
            if input_text.value:
                user_proxy.send(
                    input_text.value,
                    user_input_agent,
                    request_reply=True
                )
                user_input_response = user_input_agent.last_message()['content']
                print(f"User Input Agent response: {user_input_response}")
                
                try:
                    constraints = json.loads(user_input_response)
                except json.JSONDecodeError:
                    print("Error processing input. Using default settings.")

            lineups = optimizer_agent.generate_lineups(constraints)
            if lineups and 'error' in lineups[0]:
                print(f"Error generating lineup(s): {lineups[0]['error']}")
            else:
                current_lineup = 0
                display(HTML(display_lineup(lineups[current_lineup])))
                update_buttons()

    generate_button.on_click(on_generate_button_clicked)

    ui = widgets.VBox([
        input_text, 
        generate_button, 
        output,
        widgets.HBox([prev_button, lineup_number, next_button])
    ])
    
    display(ui)

# Initialize agents and UI
llm_config = {
    "model": "gpt-4o-mini",
    "api_key": os.environ.get("OPENAI_API_KEY"),
    "temperature": 0,
    "seed": 42,
}

user_input_agent = autogen.AssistantAgent(
    name="UserInputAgent",
    llm_config=llm_config,
    system_message="""You are an agent that processes user requests for fantasy football lineups.
    Parse the user's input and convert it into a structured format for lineup optimization.
    If the input is empty or doesn't contain specific constraints, return an empty dictionary.
    Use the following examples as a guide:

    Input: "I want Justin Jefferson in my lineup"
    Output: {"must_include": ["justinjefferson"]}

    Input: "I want a stack with Stroud and Collins"
    Output: {"must_include": ["cjstroud", "nicocollins"]}

    Input: "Generate me 2 unique lineups"
    Output: {"num_lineups": 2}

    Input: "" (empty input)
    Output: {}

    Provide your output as a valid JSON object."""
)

user_proxy = autogen.UserProxyAgent(
    name="User",
    human_input_mode="NEVER",
    max_consecutive_auto_reply=0
)

optimizer_agent = LineupOptimizerAgent(
    name="LineupOptimizer",
    data_file='data/merged_fantasy_football_data.csv',
    llm_config=llm_config
)

In [ ]:
# Create and display the UI
create_fantasy_football_ui(optimizer_agent, user_input_agent, user_proxy)